# IMPORTS

In [ ]:
import requests        # para llamadas http/https
import time            # generalista para tratar tiempos
import inspect, sys    # para obtener el nombre de la funcion desde dentro de la funcion
import pickle          # para guardar objetos a disco en binario
import matplotlib.pyplot as plt  # para graficar
import os              # para hacer listados de ficheros

# TOKEN

In [ ]:
token = '' # Mi API token de darwinex

# FUNCIONES

In [ ]:
def save_file(file_prefix):
    fichero = file_prefix + datetime.now().strftime("%Y-%m-%dT%H.%M.%S") + '.bin'
    with open(fichero, 'wb') as filehandle:
        pickle.dump(s, filehandle)
    return(fichero)

In [ ]:
def load_file(file_prefix):
    path = os.getcwd()
    files = []
    for i in os.listdir(path):
        if os.path.isfile(os.path.join(path,i)) and file_prefix in i:
            files.append(i)   
    if len(files[-1:]) > 0:
        with open((files[-1]), 'rb') as filehandle:
            k = pickle.load(filehandle)
        return k,path+'/'+files[-1]
    else: 
        return -1,-1

In [ ]:
def get_darwins_info(force_load = False):
    index = 0
    this_fun = str(sys._getframe().f_code.co_name)
    
    if force_load == False:
        ss,f = load_file(this_fun)
        if ss != -1:
            print ('Cargada info de Darwins de ',f)
            return(ss)
        
    url = 'https://api.darwinex.com/darwininfo/1.5/products?page=' + str(index)
    headers = {'Authorization': 'Bearer ' + str(token)}
    r = requests.get(url, headers=headers)
    is_last = r.json()['last']
    s = r.json()['content']
    index = index + 1
    print('(',index,') Procesando darwins ',r.json()['content'][0]['productName'],' a ', r.json()['content'][49]['productName'])
    while is_last != True:
        url = 'https://api.darwinex.com/darwininfo/1.5/products?page=' + str(index)
        r = requests.get(url, headers=headers)
        is_last = r.json()['last']
        s.extend(r.json()['content'])
        print('(',index,') Procesando darwins ',r.json()['content'][0]['productName'],' a ', r.json()['content'][len(r.json()['content'])-1]['productName'])
        if index % 10 == 0:
            print('Durmiendo 60 segundos...')
            time.sleep(60)
        index = index + 1
    print('(',save_file(this_fun),') Descarga de Darwins finalizada')
    return(s)


In [ ]:
def badge_id(badge):
    switcher = {
        0: "Ex score",
        1: "Mc score",
        2: "Rs score",
        3: "Ra score",
        4: "Os score",
        5: "Cs score",
        6: "R+ score",
        7: "R- score",
        8: "Dc score",
        9: "La score",
        10: "Pf score",
        11: "Cp score",
        12: "D-Score",
        "Ex score" : 0,
        "Mc score" : 1,
        "Rs score" : 2,
        "Ra score" : 3,
        "Os score" : 4,
        "Cs score" : 5,
        "R+ score" : 6,
        "R- score" : 7,
        "Dc score" : 8,
        "La score" : 9,
        "Pf score" : 10,
        "Cp score" : 11,
        "D-Score" : 12
    }
    return (switcher.get(badge, "Invalid badge"))

In [ ]:
def plot_darwin(darwin, badge = 'D-Score'):
    dates = [item[0] for item in darwin['badges']]
    dates[:] = [time.strftime('%Y-%m-%d',time.gmtime(x/ 1000.))  for x in dates]
    dates[:] = [datetime.strptime(x,'%Y-%m-%d')  for x in dates]
    dscores= []
    for i in range(len(darwin['badges'])):
        dscores.append(darwin['badges'][i][2][badge_id(badge)])
    plt.plot(dates,dscores) 
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
def get_badges(darwins, plot_on_the_fly = False, force_load = False):
    headers = {'Authorization': 'Bearer ' + str(token)}
    this_fun = str(sys._getframe().f_code.co_name)
    
    if force_load == False:
        s,f = load_file(this_fun)
        if s != -1:
            print ('Cargada info de Badges de Darwins de ',f)
            return(s)
        
    i = 1
    for darwin in darwins:
        print('(',i,') Darwin:',darwin['shortName'])
        url = 'https://api.darwinex.com/darwininfo/1.5/products/' + darwin['productName'] + '/history/badges'
        r = requests.get(url, headers=headers)
        darwins[i-1].update({'badges' : r.json()})
        if plot_on_the_fly:
            plot_darwin(darwin)
        if i % 10 == 0:
            print('Durmiendo 60 segundos...')
            time.sleep(60)
        i = i+1
    print('(',save_file(this_fun),') Descarga de Darwin Badges finalizada')
    return(darwins)